In [1]:
import os
import sys
home_dir = "../"
sys.path.append(home_dir)

import torch
import pandas as pd
import numpy as np
from Bio import SeqIO
import pyfastx

In [2]:
cols = ["FA-DOMID", "FA-PDBID", "FA-PDBREG", "FA-UNIID", "FA-UNIREG", "SF-DOMID", "SF-PDBID", "SF-PDBREG", "SF-UNIID", "SF-UNIREG", "SCOPCLA"]
scop_df = pd.read_csv(home_dir+"data/SCOP/downloads/scop-cla-latest.txt", sep=" ", comment="#", names=cols)
scop_df

,FA-DOMID,FA-PDBID,FA-PDBREG,FA-UNIID,FA-UNIREG,SF-DOMID,SF-PDBID,SF-PDBREG,SF-UNIID,SF-UNIREG,SCOPCLA
0,8045703,3H8D,C:1143-1264,Q64331,1143-1264,8091604,3H8D,C:1143-1264,Q64331,1143-1264,"TP=1,CL=1000003,CF=2001470,SF=3002524,FA=4004627"
1,8094330,6J56,A:1158-1282,Q9UM54,1167-1291,8094331,6J56,A:1158-1282,Q9UM54,1167-1291,"TP=1,CL=1000003,CF=2001470,SF=3002524,FA=4004627"
2,8017835,3FKQ,A:1-116,D0VX10,1-116,8017836,3FKQ,A:1-116,D0VX10,1-116,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4003986"
3,8021315,1XHF,A:2-122,P0A9Q1,2-122,8033695,1XHF,A:2-122,P0A9Q1,2-122,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4003632"
4,8021787,1Y7P,B:79-215,O28869,79-215,8034167,1Y7P,B:79-215,O28869,79-215,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4002481"
...,...,...,...,...,...,...,...,...,...,...,...
36895,8022870,1SCJ,B:307-377,P04189,36-106,8035250,1SCJ,B:307-377,P04189,36-106,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001246"
36896,8027500,1T1E,A:12-188,Q8RR56,12-188,8039879,1T1E,A:12-188,Q8RR56,12-188,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001246"
36897,8027593,1JQG,A:4P-100P,O97389,18-109,8039972,1JQG,A:4P-100P,O97389,18-109,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001250"
36898,8028119,1KN6,A:4-76,P63239,31-103,8040498,1KN6,A:4-76,P63239,31-103,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4002630"


In [3]:
def separate_scopclass(row):
    x = row["SCOPCLA"]
    tp, cl, cf, sf, fa = x.split(",") # ie. TP=1,CL=1000001,CF=2000893,SF=3001451,FA=4004596
    tp, cl, cf, sf, fa = str(tp[3:]), str(cl[3:]), str(cf[3:]), str(sf[3:]), str(fa[3:])
    return tp, cl, cf, sf, fa

scop_df[["TP", "CL", "CF", "SF", "FA"]] = scop_df.apply(separate_scopclass, axis=1, result_type="expand")
scop_df

,FA-DOMID,FA-PDBID,FA-PDBREG,FA-UNIID,FA-UNIREG,SF-DOMID,SF-PDBID,SF-PDBREG,SF-UNIID,SF-UNIREG,SCOPCLA,TP,CL,CF,SF,FA
0,8045703,3H8D,C:1143-1264,Q64331,1143-1264,8091604,3H8D,C:1143-1264,Q64331,1143-1264,"TP=1,CL=1000003,CF=2001470,SF=3002524,FA=4004627",1,1000003,2001470,3002524,4004627
1,8094330,6J56,A:1158-1282,Q9UM54,1167-1291,8094331,6J56,A:1158-1282,Q9UM54,1167-1291,"TP=1,CL=1000003,CF=2001470,SF=3002524,FA=4004627",1,1000003,2001470,3002524,4004627
2,8017835,3FKQ,A:1-116,D0VX10,1-116,8017836,3FKQ,A:1-116,D0VX10,1-116,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4003986",1,1000002,2000016,3001156,4003986
3,8021315,1XHF,A:2-122,P0A9Q1,2-122,8033695,1XHF,A:2-122,P0A9Q1,2-122,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4003632",1,1000002,2000016,3001156,4003632
4,8021787,1Y7P,B:79-215,O28869,79-215,8034167,1Y7P,B:79-215,O28869,79-215,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4002481",1,1000002,2000016,3001156,4002481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36895,8022870,1SCJ,B:307-377,P04189,36-106,8035250,1SCJ,B:307-377,P04189,36-106,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001246",1,1000003,2000014,3001155,4001246
36896,8027500,1T1E,A:12-188,Q8RR56,12-188,8039879,1T1E,A:12-188,Q8RR56,12-188,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001246",1,1000003,2000014,3001155,4001246
36897,8027593,1JQG,A:4P-100P,O97389,18-109,8039972,1JQG,A:4P-100P,O97389,18-109,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001250",1,1000003,2000014,3001155,4001250
36898,8028119,1KN6,A:4-76,P63239,31-103,8040498,1KN6,A:4-76,P63239,31-103,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4002630",1,1000003,2000014,3001155,4002630


### Applying filters

In [4]:
node_desc = []
with open(home_dir+"data/SCOP/downloads/scop-des-latest.txt") as f:
    for line in f.readlines():
        if line.startswith("#"): continue
        line = line.strip()
        node_id, node_name = line.split(" ")[0], " ".join(line.split(" ")[1:])

        node_desc.append({"NODE_ID": node_id, "NODE_NAME": node_name})
        # break

node_desc_df = pd.DataFrame.from_dict(node_desc)
node_desc_df

,NODE_ID,NODE_NAME
0,1,Globular proteins
1,2,Membrane proteins
2,3,Fibrous proteins
3,4,Non-globular/Intrinsically unstructured proteins
4,1000000,All alpha proteins
...,...,...
10335,4007841,HHM C-terminal domain-like
10336,4007842,Proton-activated chloride channel TMEM206-like
10337,4007843,Centrosomal protein CEP44-like
10338,4007844,SPD2 domain-like


In [5]:
# excluding Rossmann folds and 4- to 8-bladed beta-propeller
rossmann_fold_domains = node_desc_df[node_desc_df["NODE_NAME"].str.contains("Rossmann fold")]
beta_propeller_4to8_domains = node_desc_df[node_desc_df["NODE_NAME"].str.contains("4-bladed beta-propeller|5-bladed beta-propeller|6-bladed beta-propeller|7-bladed beta-propeller|8-bladed beta-propeller")]
exclude_folds = pd.concat([rossmann_fold_domains, beta_propeller_4to8_domains])
exclude_folds

,NODE_ID,NODE_NAME
14,2000005,Canonical Rossmann fold
131,2000148,SDR-type extended Rossmann fold
132,2000149,OCD-type extended Rossmann fold
134,2000151,Variant Rossmann fold
135,2000152,CoA-binding domain-type Rossmann fold
136,2000154,6PGDH-type extended Rossmann fold
919,2000959,4-bladed beta-propeller
932,2000972,5-bladed beta-propeller
942,2000982,6-bladed beta-propeller
973,2001013,7-bladed beta-propeller


In [6]:
# scop_df[scop_df["CF"]=="2001061"]
# exclude_folds["NODE_ID"].to_list()
print("#-of datapoints classified into Rossmann folds or 4- to 8- beta-propellers:", scop_df[scop_df["CF"].isin(exclude_folds["NODE_ID"].to_list())].shape)

filtered_scop_df = scop_df[~scop_df["CF"].isin(exclude_folds["NODE_ID"].to_list())]
filtered_scop_df.shape

#-of datapoints classified into Rossmann folds or 4- to 8- beta-propellers: (1313, 16)


(35587, 16)

In [7]:
# excluding datapoints where Family and Superfamily denoting sequence are different
print("#-of datapoints where Family and Superfamily denoting sequence are different:", filtered_scop_df[filtered_scop_df["FA-PDBREG"]!=filtered_scop_df["SF-PDBREG"]].shape)

filtered_scop_df = filtered_scop_df[filtered_scop_df["FA-PDBREG"]==filtered_scop_df["SF-PDBREG"]]
filtered_scop_df.shape

#-of datapoints where Family and Superfamily denoting sequence are different: (4181, 16)


(31406, 16)

In [8]:
# excluding datapoints where Family and Superfamily denoting PDB ids are different
print("#-of datapoints where Family and Superfamily denoting PDB ids are different:", filtered_scop_df[filtered_scop_df["FA-PDBID"]!=filtered_scop_df["SF-PDBID"]].shape)

filtered_scop_df = filtered_scop_df[filtered_scop_df["FA-PDBID"]==filtered_scop_df["SF-PDBID"]]
filtered_scop_df.shape

#-of datapoints where Family and Superfamily denoting PDB ids are different: (0, 16)


(31406, 16)

In [9]:
# excluding datapoints Family/Superfamily denoting sequence have multiple spans
def have_one_span(x):
    return len(x.split(",")) == 1 # A:37-243,A:353-401

print("#-of datapoints where Family denoting sequence have multiple spans:", filtered_scop_df[~filtered_scop_df["FA-PDBREG"].apply(have_one_span)].shape)
filtered_scop_df = filtered_scop_df[filtered_scop_df["FA-PDBREG"].apply(have_one_span)]
print(filtered_scop_df.shape)

print("#-of datapoints where Superfamily denoting sequence have multiple spans:", filtered_scop_df[~filtered_scop_df["SF-PDBREG"].apply(have_one_span)].shape)
filtered_scop_df = filtered_scop_df[filtered_scop_df["SF-PDBREG"].apply(have_one_span)]

filtered_scop_df.shape

#-of datapoints where Family denoting sequence have multiple spans: (330, 16)
(31076, 16)
#-of datapoints where Superfamily denoting sequence have multiple spans: (0, 16)


(31076, 16)

In [10]:
# Filtering Family representative sequences. 
# Since after filtering Family and Superfamily representative sequeces come from the same PDBid+chainid+span, so do not need to filter Superfamily representative sequences
filter_dom = "FA" # SF, FA
fa = pyfastx.Fasta(home_dir+f"data/SCOP/downloads/scop_{filter_dom.lower()}_represeq_lib_latest.fa")
seq_ids_set = set(fa.keys())
family_domids_set = set(filtered_scop_df[f"{filter_dom}-DOMID"].apply(str).to_list())
ids = seq_ids_set.intersection(family_domids_set) # these ids will be considered only
print(len(ids), len(family_domids_set))

31076 31076


In [11]:
# saving the filtered sequences for clustering in a fasta format
# ~ 9min
# do not run it unless needed
with open(home_dir + f"data/SCOP/processed/scop_{filter_dom.lower()}_represeq.fa", mode="w") as f:
    for i, id in enumerate(ids):
        # print(fa[id].description) # id, name, seq, description
        f.write(f">{fa[id].description}\n{fa[id].seq}\n")
        # break
        if i%1000==0: print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


In [12]:
# converting seqs into Dataframe format
# ~ 9min
data = []
for i, id in enumerate(ids):
    data.append({"id":int(id), "seq":str(fa[id].seq).strip().upper()}) # "description":str(fa[id].description), 
    if i%1000==0: print(i)
    # break

seq_df = pd.DataFrame.from_dict(data)
seq_df

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


,id,seq
0,8028535,FRTKPAPVDPSLQHEIEQFYYWEAKLLNDRRFQEWFDLLAEDIHYF...
1,8106359,EVKIVVDRDPVKTSFEEWARPGHFSRTLAKGPDTTTWIWNLHADAH...
2,8025302,RENALLSSSLWVNVALAGIAILVFVYMGRTIRPGRPRLIWGATLMI...
3,8019896,GLRSRAWFKLDEIQQSDKLFKPGMTVVDLGAAPGGWSQYVVTQIGG...
4,8072176,PQITLWQRPIVTIKIGGQLKEALLDTGADDTVLEDINLPGRWKPKM...
...,...,...
31071,8027259,LKEVWEELDGLDPNRFNPKTFFILHDINSDGVLDEQELEALFTKEL...
31072,8021720,KLASTMEGRVEQLAEQRQVIEAGGGERRVEKQHSQGKQTARERLNN...
31073,8020429,SLQGKIVFAVGGAPNEIEYWKGVIAEFEKKYPGVTVELKRQATDTE...
31074,8050267,PNNDMKLFDQAGYYVLRNGWTPASTVMILSNNKSNDASNSLSAYSH...


In [13]:
filtered_scop_with_seq_df = filtered_scop_df.merge(right=seq_df, how="inner", left_on="FA-DOMID", right_on="id")
filtered_scop_with_seq_df.drop(columns=["id"], inplace=True)
filtered_scop_with_seq_df.reset_index(drop=True, inplace=True)
filtered_scop_with_seq_df

,FA-DOMID,FA-PDBID,FA-PDBREG,FA-UNIID,FA-UNIREG,SF-DOMID,SF-PDBID,SF-PDBREG,SF-UNIID,SF-UNIREG,SCOPCLA,TP,CL,CF,SF,FA,seq
0,8045703,3H8D,C:1143-1264,Q64331,1143-1264,8091604,3H8D,C:1143-1264,Q64331,1143-1264,"TP=1,CL=1000003,CF=2001470,SF=3002524,FA=4004627",1,1000003,2001470,3002524,4004627,DMKRQQRFFRIPFIRPADQYKDPQNKKKGWWYAHFDGPWIARQMEL...
1,8094330,6J56,A:1158-1282,Q9UM54,1167-1291,8094331,6J56,A:1158-1282,Q9UM54,1167-1291,"TP=1,CL=1000003,CF=2001470,SF=3002524,FA=4004627",1,1000003,2001470,3002524,4004627,RQREIEMNRQQRFFRIPFIRPADQYKDPQSKKKGWWYAHFDGPWIA...
2,8017835,3FKQ,A:1-116,D0VX10,1-116,8017836,3FKQ,A:1-116,D0VX10,1-116,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4003986",1,1000002,2000016,3001156,4003986,MKIKVALLDKDKEYLDRLTGVFNTKYADKLEVYSFTDEKNAIESVK...
3,8021315,1XHF,A:2-122,P0A9Q1,2-122,8033695,1XHF,A:2-122,P0A9Q1,2-122,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4003632",1,1000002,2000016,3001156,4003632,QTPHILIVEDELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEYD...
4,8021787,1Y7P,B:79-215,O28869,79-215,8034167,1Y7P,B:79-215,O28869,79-215,"TP=1,CL=1000002,CF=2000016,SF=3001156,FA=4002481",1,1000002,2000016,3001156,4002481,SFERVFGKRVIILGGGALVSQVAIGAISEADRHNLRGERISVDTMP...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31071,8022870,1SCJ,B:307-377,P04189,36-106,8035250,1SCJ,B:307-377,P04189,36-106,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001246",1,1000003,2000014,3001155,4001246,EKKYIVGFKQTMSAMSSAKKKDVISEKGGKVQKQFKYVNAAAATLD...
31072,8027500,1T1E,A:12-188,Q8RR56,12-188,8039879,1T1E,A:12-188,Q8RR56,12-188,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001246",1,1000003,2000014,3001155,4001246,EKREVLAGHARRQAPQAVDKGPVTGDQRISVTVVLRRQRGDELEAH...
31073,8027593,1JQG,A:4P-100P,O97389,18-109,8039972,1JQG,A:4P-100P,O97389,18-109,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4001250",1,1000003,2000014,3001155,4001250,HEIYDGHAVYQVDVASMDQVKLVHDFENDLMLDVWSDAVPGRPGKV...
31074,8028119,1KN6,A:4-76,P63239,31-103,8040498,1KN6,A:4-76,P63239,31-103,"TP=1,CL=1000003,CF=2000014,SF=3001155,FA=4002630",1,1000003,2000014,3001155,4002630,FVNEWAAEIPGGQEAASAIAEELGYDLLGQIGSLENHYLFKHKSHP...


In [14]:
filtered_scop_with_seq_df.to_csv(home_dir+"data/SCOP/processed/SCOP_with_seq.tsv", sep="\t", header=True, index=False) # SCOP data with family representative sequence